In [3]:
import sys
print(sys.version)

3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]


In [4]:
# Gather .csv files. 
# Join R3F002, R3F015, and DealerCurrency. 
# Final table in tidy format.
# Note: The code has some bugs. Also, part of the code needs to be customised to your environment and storage place of the csv files in your desktop.

In [0]:
# Change code as needed
df_R3F002 = ks.read_csv('/dim-p/SE_Sample_Data.csv',sep = "_")
df_R3F002.createOrReplaceTempView("SE_Sample_Data")

In [0]:
# Change code as needed
df_R3F002 = spark.sql("""
  SELECT
    CONCAT_WS('_', RIGHT(CONCAT_WS('', '0000000000', BDLNO), 10), RIGHT(CONCAT_WS('', '0000000000', BDSTS), 10)) as DealerDistrictKey,
    BNGSI as InfluenceOnServiceIndex,
    BDLNO as Dealer,
    BDSTS as District,
    BCDMO as MovementCode,   
    BCDPU as PurchaseCode,
    BCLFR as FrequencyClass,
    BCLPR as PriceClass ,
    BCMPC as CompanyCode,
    BDTFC as DateOfForecastRecalc,
    BFCMD0 as ForecastMethodM00,
    BFCMD1 as ForecastMethodM01,
    BFCMD2 as ForecastMethodM02,
    BFCMD3 as ForecastMethodM03,
    BFCMD4 as ForecastMethodM04,
    BFCMD5 as ForecastMethodM05,
    BFCMD6 as ForecastMethodM06,
    BFCMD7 as ForecastMethodM07,
    BFCMD8 as ForecastMethodM08,
    BFCMD9 as ForecastMethodM09,
    BFCMD10 as ForecastMethodM10,
    BFCMD11 as ForecastMethodM11,
    BFCMD12 as ForecastMethodM12,
    BFFCO as ForecastFreezeCounter,
    BHSNP as HighSalesNoPeriodCounter,
    BINCD as InfoCode,
    (BNP1+BNP2+BNP3+BNP4+BNP5+BNP6+BNP7+BNP8+BNP9+BNP10+BNP11+BNP12) as PicksR12,
    BNP0 as NoOfPicksPerM00,
    BNP1 as NoOfPicksPerM01,
    BNP2 as NoOfPicksPerM02,
    BNP3 as NoOfPicksPerM03,
    BNP4 as NoOfPicksPerM04,
    BNP5 as NoOfPicksPerM05,
    BNP6 as NoOfPicksPerM06,
    BNP7 as NoOfPicksPerM07,
    BNP8 as NoOfPicksPerM08,
    BNP9 as NoOfPicksPerM09,
    BNP10 as NoOfPicksPerM10,
    BNP11 as NoOfPicksPerM11,
    BNP12 as NoOfPicksPerM12,
    BNP13 as NoOfPicksPerM13,
    BNPC as NewPartCode,
    BOPP  as OrderPoint, 
    BPCTXT as PurchaseCodeText,
    BPPRF as PartPrefix,
    BPRIS as Price,
    BPRTN as PartNumber,
    BQFC as ForecastPM00,
    BQFC1 as ForecastPM01,
    BQFC2 as ForecastPM02,
    BQFC3 as ForecastPM03,
    BQFC4 as ForecastPM04,
    BQFC5 as ForecastPM05
    BQFC6 as ForecastPM06,
    BQFC7 as ForecastPM07,
    BQFC8 as ForecastPM08,
    BQFC9 as ForecastPM09,
    BQFC10 as ForecastPM10,
    BQFC11 as ForecastPM11,
    BQFC12 as ForecastPM12,
    (BQFC*BPRIS) as ForecastCurrentValue,
    ((BQS1+BQS2+BQS3+BQS4+BQS5+BQS6+BQS7+BQS8+BQS9+BQS10+BQS11+BQS12)/12) as AvgSalesR12
    BQS0 as SalesPM00,         
    BQS1 as SalesPM01,
    BQS2 as SalesPM02,
    BQS3 as SalesPM03,
    BQS4 as SalesPM04,
    BQS5 as SalesPM05,
    BQS6 as SalesPM06,
    BQS7 as SalesPM07,
    BQS8 as SalesPM08,
    BQS9 as SalesPM09,
    BQS10 as SalesPM10,
    BQS11 as SalesPM11,
    BQS12 as SalesPM12,
    BQS13 as SalesPM13,
    BRPED as ReplacedCode,
    BRPNG as ReplacingCode, 
    BSTBA as StockBalanceQty,
    BSUGR as SupplierGroup,
    BTRSL as TrackingSignal,
  FROM 
    R3F002
  WHERE   
    (BQFC+BQFC1+BQFC2+BQFC3+BQFC4+BQFC5+BQFC6+BQFC7+BQFC8+BQFC9+BQFC10+BQFC11+BQFC12)>0
    AND (BQS0+BQS1+BQS2+BQS3+BQS4+BQS5+BQS6+BQS7+BQS8+BQS9+BQS10+BQS11+BQS12)>0
    AND BSUGR IN ('0','1','2','3','4','5','6','7','8')
    AND BPPRF IN ('VO','RT','MK','VOP','BRA')
""")
df_R3F002.createOrReplaceTempView("tv_R3F002")

In [0]:
# Change code as needed
df_R3F015 = ks.read_csv('/mnt/dim-followup/ForecastTool/R3F015.csv',sep = ";")
df_R3F015.createOrReplaceTempView("R3F015")
df_R3F015 = spark.sql("""
  SELECT
    CONCAT_WS('_',RIGHT(CONCAT_WS('', '0000000000', PDLNO), 10), RIGHT(CONCAT_WS('', '0000000000', PDSTS), 10)) as DealerDistrictKey,
    PDLNO as Dealer,
    PDSTS as District,
    PCUR as CurrencyCode,
    PLPA as LPADealer,
    PCCODE as CountryCode,
    If(PISFL='Y', 'Y', 'N') as IgnoreSeasonalTest
  FROM 
    R3F015
  WHERE
    length(PCCODE)>0 AND  length(PCUR)>0 AND PLIFT ='Y'
""")
df_R3F015.createOrReplaceTempView("tv_R3F015")

In [0]:
# Change code as needed
df_DealerCurrency = ks.read_csv('/mnt/dim-followup/ForecastTool/DealerCurrency.csv',sep = "|")
df_DealerCurrency = df_DealerCurrency.to_spark()
df_DealerCurrency.createOrReplaceTempView("DealerCurrency")
df_DealerCurrency = spark.sql("""
  SELECT
    DealerCurrency.PCUR as CurrencyCode,
    Replace(DealerCurrency.`Currency_Rate`, ',', '.') as CurrencyRate
  FROM 
    DealerCurrency
""")
df_DealerCurrency.createOrReplaceTempView("tv_DealerCurrency")

In [0]:
# Change code as needed
df_R3F002_R3F015_DealerCurrency = spark.sql("""
SELECT 
 tv_R3F002.DealerDistrictKey,
 CONCAT_WS('_', tv_R3F002.DealerDistrictKey, RIGHT(CONCAT_WS('', '0000000000', tv_R3F002.PartNumber), 10)) as DealerDistrictPartNumberKey,
 tv_R3F002.InfluenceOnServiceIndex,
 tv_R3F002.Dealer,
 tv_R3F002.District,
 tv_R3F002.MovementCode,
 tv_R3F002.PurchaseCode,
 tv_R3F002.FrequencyClass,
 tv_R3F002.PriceClass,
 tv_R3F002.CompanyCode,
 tv_R3F002.DateOfForecastRecalc,
 tv_R3F002.ForecastMethodM00,
 tv_R3F002.ForecastMethodM01,
 tv_R3F002.ForecastMethodM02,
 tv_R3F002.ForecastMethodM03,
 tv_R3F002.ForecastMethodM04,
 tv_R3F002.ForecastMethodM05,
 tv_R3F002.ForecastMethodM06,
 tv_R3F002.ForecastMethodM07,
 tv_R3F002.ForecastMethodM08,
 tv_R3F002.ForecastMethodM09,
 tv_R3F002.ForecastMethodM10,
 tv_R3F002.ForecastMethodM11,
 tv_R3F002.ForecastMethodM12,
 tv_R3F002.ForecastFreezeCounter,
 tv_R3F002.HighSalesNoPeriodCounter,
 tv_R3F002.InfoCode,
 tv_R3F002.PicksR12,
 tv_R3F002.NoOfPicksPerM00,
 tv_R3F002.NoOfPicksPerM01,
 tv_R3F002.NoOfPicksPerM02,
 tv_R3F002.NoOfPicksPerM03,
 tv_R3F002.NoOfPicksPerM04,
 tv_R3F002.NoOfPicksPerM05,
 tv_R3F002.NoOfPicksPerM06,
 tv_R3F002.NoOfPicksPerM07,
 tv_R3F002.NoOfPicksPerM08,
 tv_R3F002.NoOfPicksPerM09,
 tv_R3F002.NoOfPicksPerM10,
 tv_R3F002.NoOfPicksPerM11,
 tv_R3F002.NoOfPicksPerM12,
 tv_R3F002.NoOfPicksPerM13,
 tv_R3F002.NewPartCode,
 tv_R3F002.OrderPoint,
 tv_R3F002.PurchaseCodeText,
 tv_R3F002.PartPrefix,
 tv_R3F002.Price,
 tv_R3F002.PartNumber,
 tv_R3F002.ForecastPM00,
 tv_R3F002.ForecastPM01,
 tv_R3F002.ForecastPM02,
 tv_R3F002.ForecastPM03,
 tv_R3F002.ForecastPM04,
 tv_R3F002.ForecastPM05,
 tv_R3F002.ForecastPM06,
 tv_R3F002.ForecastPM07,
 tv_R3F002.ForecastPM08,
 tv_R3F002.ForecastPM09,
 tv_R3F002.ForecastPM10,
 tv_R3F002.ForecastPM11,
 tv_R3F002.ForecastPM12,
 tv_R3F002.ForecastCurrentValue,
 tv_R3F002.AvgSalesR12,
 tv_R3F002.SalesPM00,
 tv_R3F002.SalesPM01,
 tv_R3F002.SalesPM02,
 tv_R3F002.SalesPM03,
 tv_R3F002.SalesPM04,
 tv_R3F002.SalesPM05,
 tv_R3F002.SalesPM06,
 tv_R3F002.SalesPM07,
 tv_R3F002.SalesPM08,
 tv_R3F002.SalesPM09,
 tv_R3F002.SalesPM10,
 tv_R3F002.SalesPM11,
 tv_R3F002.SalesPM12,
 tv_R3F002.SalesPM13,
 tv_R3F002.ReplacedCode,
 tv_R3F002.ReplacingCode,
 tv_R3F002.StockBalanceQty,
 tv_R3F002.SupplierGroup,
 tv_R3F002.TrackingSignal,
 tv_R3f015.CurrencyCodes,
 tv_R3f015.LPADealer,
 tv_R3f015.CountryCode,
 tv_R3f015.IgnoreSeasonalTest,
 tv_DealerCurrency.CurrencyRate
FROM 
    tv_R3F002
JOIN 
    tv_R3f015 
JOIN 
    tv_DealerCurrency
ON 
    tv_R3F002.DealerDistrictKey = tv_R3F015.DealerDistrictKey
AND
    tv_R3F015.CurrencyCode = tv_DealerCurrency.CurrencyCode
""")
df_R3F002_R3F015_DealerCurrency.createOrReplaceTempView("tv_R3F002_R3F015_DealerCurrency")

In [0]:
# Change code as needed
# Unpivoting SalesPMs as (SalesMonthU, SalesUnits),
# Calculate SalesPMs * Price * CurrencyRate as SalesSEK, 
# Unpivoting ForecastPMs AS (ForecastMonthU, ForecastUnits), 
# Calculate ForecastPMs * Price * CurrencyRate as ForecastSEK, 
# Unpivoting ForecastMethod as (ForecastMethodMonthU, ForecastMethod), 
# Unpivoting NoOfPicksPerMs as (NoOfPicksMonth, NoOfPicks)
# It is ok to do not have the minus 13 because we don't have the forcast -13
df_R3F002_R3F015_DealerCurrency_Unpivot = spark.sql(""" 
  SELECT 

  FROM 
    tv_R3F002_R3F015_DealerCurrency
""")